# Project Scenario

Our corporate client Used-Car Online Pty Ltd (UCO) is asking us to provide a comprehensive data analysis of the used car online market in the USA, according to the dataset file named ‘Vehicles.csv’. UCO requires from us the following:

* Demonstrate a clear data frame to list ['id', 'region', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'description', 'county', 'state', 'lat', 'long', 'posting_date’]


* The top 10 state ranking by the most cars listed and its correlation with population each state - Ina


* Correlation between odometer (kms run) and price of the car – Dhiraj


* The different types of vehicles listed in US from the year 2000 to 2022 – Anjana


* Observations based on comparing the Color of the Vehicles listed with the number of listings, prices of listings and manufacturer – Deon


* What is the most profitable way to sell cars online? - Tim

In [1]:
# Dependencies
import os
import pandas as pd
import geoviews as gv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import numpy as np

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'geoviews'

# Clean Dataset 'vehicles.csv'

Dataset Source from: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data?resource=download

In [ ]:
# Get raw dataset 'vehicles.csv'
vehicles_data = os.path.join("Resources", "vehicles.csv")
vehicles_df = pd.read_csv(vehicles_data)
vehicles_df.head()

In [ ]:
# Columns in raw dataset 
vehicles_df.columns

In [ ]:
# Delete some unuseful data columns
vehicles_column = vehicles_df.drop(["url", "region_url", "image_url", "description", "county", "VIN"], axis = 1)
vehicles_column.head()

In [ ]:
# Numbers of used-car listed in the dataset without dropna
vehicles_column.count()

In [ ]:
# Delet duplicate series in the dataset
vehicles_duplicate = vehicles_column[vehicles_column.duplicated(subset = ["id"])]
vehicles_duplicate["id"].unique()

In [ ]:
# Dropna in the dataset
vehicles_na = vehicles_column.dropna(how = "any")
vehicles_na

In [ ]:
# Numbers of used-car listed after data clean
vehicles_na.count()

In [ ]:
# Find used-car made after year 2000
vehicles_sorted = vehicles_na[(vehicles_na["year"] >= 2000) & (vehicles_na["year"] <= 2022)]
vehicles_sorted.head()

In [ ]:
# Remove the outlier price
vehicles_price_out = vehicles_sorted[((vehicles_sorted["price"] != 1234) &
                                  (vehicles_sorted["price"] != 12345) &
                                  (vehicles_sorted["price"] != 123456) &
                                  (vehicles_sorted["price"] != 1234567) &
                                  (vehicles_sorted["price"] != 12345678) &
                                  (vehicles_sorted["price"] != 123456789) &
                                  (vehicles_sorted["price"] != 1234567890))]
vehicles_price_out.head()

# Remove the outlier price
vehicles_price = vehicles_price_out[((vehicles_sorted["price"] > 1000) & (vehicles_sorted["price"] < 200000))]
vehicles_price.head()

In [ ]:
# Create a dataset after data clean with dropna, duplicate and unique function
vehicles_cleaned_df = vehicles_price.reset_index(drop = True)
vehicles_cleaned_df.head()

In [ ]:
# Numbers of used-car listed after data clean & made in 2000 and afterward
vehicles_cleaned_df.count()

In [ ]:
# saved the cleaned dataset into output_data folder and ready for data analysis
vehicles_cleaned_df.to_csv("output_data/vehicles_cleaned.csv")

# Section 1

# The top 10 state ranking by number of vehicle listed and their corrlation with the population of each state - Ina

EDA Methodology:

* upper(): to capitalize the state abbreviation in the column of ‘state’
* groupby(): to create a new data frame ‘geo_df’ with ‘state’ as an index and 3 columns ‘lat’, ‘long’ and ‘Total_Listed’
* sort_values(by=‘Total_Listed’) and head(10): to find the top 10 states ranking by the number of vehicles listed
* merge(), loc() & for loop: to merge cleaned vehicle data with USA population & state abbreviation datasets
* stats.pearsonr(),stats.linregress(): to calculate correlation coefficient and linear regression

In [ ]:
# Upper the state abbreviation
vehicles_cleaned_df['state'] = vehicles_cleaned_df['state'].str.upper()
vehicles_cleaned_df.head()

In [ ]:
# Create a new dataframe geo_df with geographic allocation lantitude and longtitude for the listed used car by groupby operations
lat = vehicles_cleaned_df.groupby(['state']).mean()['lat']
geo_df = lat.to_frame()
long = vehicles_cleaned_df.groupby(['state']).mean()['long']
list_number = vehicles_cleaned_df.groupby(['state']).count()['id']
geo_df['long'] = long.values
geo_df['Total_Listed'] = list_number.values
geo_df.head(10)

In [ ]:
# Count total number of states in the data frame
geo_df.count()['Total_Listed']

In [ ]:
# Sort the geo_df and find the top 10 states ranking by number of vehicle listed
geo_df = geo_df.sort_values(by=['Total_Listed'], ascending=False)
geo_df_top10 = geo_df.head(10)
geo_df_top10

In [ ]:
# Calculate the % of Top 10 & Top 3 listed vehicles with the total number of listed vehicles
Top10_vs_all = geo_df_top10.sum()['Total_Listed'] / geo_df.sum()['Total_Listed'] * 100
"{:,}%".format(round(Top10_vs_all ,2))

Top3_vs_Top10 = (geo_df_top10.iloc[0]['Total_Listed'] + geo_df_top10.iloc[1]['Total_Listed'] \
    + geo_df_top10.iloc[0]['Total_Listed']) / geo_df_top10.sum()['Total_Listed'] * 100 
"{:,}%".format(round(Top3_vs_Top10,2))

Top3_vs_all = (geo_df_top10.iloc[0]['Total_Listed'] + geo_df_top10.iloc[1]['Total_Listed'] \
    + geo_df_top10.iloc[0]['Total_Listed']) / geo_df.sum()['Total_Listed'] * 100
"{:,}%".format(round(Top3_vs_all,2))

print(f'The top 10 state ranking by their number of listed vehicles shows in the fig Top10_Listed.png.')
print(f'The top 10 state has occupied {"{:,}%".format(round(Top10_vs_all ,2))} of the total number of liested vehicles.')
print(f'Especially {geo_df_top10.index[0:3].values} listed {"{:,}%".format(round(Top3_vs_Top10,2))} of total number of top 10 vehicles.')
print(f'The top 3 states also held {"{:,}%".format(round(Top3_vs_all,2))} vehicles vs number listed for all USA states.')

In [ ]:
# Bar chart for Top10 list car and those states
geo_df_top10.plot.bar(y='Total_Listed',rot=0)
plt.xlabel('Top 10 States')
plt.ylabel('Total Listed Vehicles')
plt.title('Top 10 states ranking by Total Listed Vehicles')

# Save the figure
plt.savefig("output_data/Top10_Listed.png")

# Summary:
#### The top 10 states have occupied more than half of the total number of listed vehicles. Especially the top 3 states California, Florida & New York State. The top 3 states share nearly one-third of listed vehicle numbers and generally consider California will be the biggest potential market followed by Floria & New York States. And the other states in the top 10 could be covered in the 2nd stage of marketing strategical development.

In [ ]:
# Geographic location for top 10 states
map_plot_1 = geo_df_top10.hvplot.points('long', 'lat', geo=True, color='red', alpha=0.2,
                       xlim=(-140, -50), ylim=(0, 55), xlabel= 'Longitude', ylabel='Latitude', tiles='OSM',  
                       size = "Total_Listed", frame_width=800)

map_plot_1

In [ ]:
print(f'The hvplot with the open map shows the geographical location of the top 10 states by their longitude and latitude. \
It is obvious to find out the top 10 states mainly located on the eastern or western coasts of USA. The size of the bubble \
reflects the total number of listed vehicles. This will be clearly identified that Califonia held the largest number of listed \
vehicles followed by Florida and New York State.')

### To run 'from bokeh.io import export_svg' please do: pip install selenium in git bash

In [ ]:
# save fig into output_data folder
from bokeh.io import export_svg
hvplot.save(map_plot_1, 'output_data/Top10_state.png')

### import a dataset regarding USA population names 'data.csv' for merging with 'Total_listed' data and complete correlation analysis
Dataset Source from: https://worldpopulationreview.com/states/state-abbreviations, 
https://worldpopulationreview.com/states

In [ ]:
# Get raw dataset USA State population list named 'data.csv'
usa_population_data = os.path.join("Resources", "data.csv")
usa_population_df = pd.read_csv(usa_population_data)
usa_population_df = usa_population_df.sort_values('state')
usa_population_df.head()

In [ ]:
# Get raw dataset USA State abbreviation list named 'data_abb.csv.csv'
usa_abb_data = os.path.join("Resources", "data_abb.csv")
usa_abb_df = pd.read_csv(usa_abb_data)
usa_abb_df.head()

In [ ]:
# Merge two data frame USA population and state abbreviation into a data frame
usa_population_df = usa_abb_df.merge(usa_population_df)
usa_population_df.head()

In [ ]:
# Merge 'Total_Listed' data into USA population dataset by state
usa_population_df['Total_Listed'] = '0'
for i in range(39):
    j = geo_df.index[i]
    usa_population_df['Total_Listed'].loc[usa_population_df['code'] == j]= geo_df.loc[geo_df.index == j]['Total_Listed'].values
usa_population_df = usa_population_df.sort_values(by='rank')
usa_population_df.head(10)

In [ ]:
# Calculate the correlation coefficient and linear regression model for number of 'Total_listed' \
# car and USA population in 2022 
x=usa_population_df['pop2022']
y=usa_population_df['Total_Listed'].astype(int)
pearsonr = st.pearsonr(x, y)
print(f'The Pearson correlation coefficient for number of Total_listed and USA population in 2022 is \
{round(pearsonr[0],2)}')

In [ ]:
# Calculate the correlation coefficient and linear regression model for number of 'Total_listed' \
# car vs USA population in 2022 
slope, intercept, r, p, std_err = st.linregress(x, y)

def linearRegression(x):
    return slope * x + intercept

linearRegression_list = list(map(linearRegression,x))

In [ ]:
# Plot out number of 'Total_listed' car vs USA population in 2022
plt.scatter(x, y,c='b')
plt.plot(x, linearRegression_list,c='r')
plt.xlabel('USA Population 2022')
plt.ylabel('Total_Listed Vehicle')
plt.title('USA Population 2022 vs. Total_Listed Vehicle')
plt.annotate(f'y = {round(slope,4)} * x + {round(intercept,2)}',(20,6000), fontsize=14)

# Save the figure
plt.savefig("output_data/population_Total_Listed.png")

plt.show()

# Summary:
#### The Pearson’s r is 0.89 and means a fairly strong positive relationship between the population and the number of listed vehicles. This reminds us to focus on developing online sales markets in the states with a larger population.

In [ ]:
# Calculate the correlation coefficient and linear regression model for number of 'Total_listed' vehicles \
# and population density of state
x=usa_population_df['densityMi']
y=usa_population_df['Total_Listed'].astype(int)
pearsonr = st.pearsonr(x, y)
print(f'The Pearson correlation coefficient for number of Total_listed car and population density in 2022 \
is {round(pearsonr[0],2)}')

In [ ]:
# Calculate the correlation coefficient and linear regression model for number of 'Total_listed' vehicles \
# vs population density of state
slope, intercept, r, p, std_err = st.linregress(x, y)

def linearRegression(x):
    return slope * x + intercept

linearRegression_list = list(map(linearRegression,x))

In [ ]:
# Plot out number of 'Total_listed' vehicles vs population density of state
plt.scatter(x, y,c='b')
plt.plot(x, linearRegression_list,c='r')
plt.xlabel('Population Density of State')
plt.ylabel('Total_Listed Vehicle')
plt.title('Population Density of State vs. Total_Listed Vehicle')
plt.annotate(f'y = {round(slope,5)} * x + {round(intercept,2)}',(20,6000), fontsize=14)

# Save the figure
plt.savefig("output_data/Population_Density_Total_Listed.png")

plt.show()

# Summary:
The Pearson’s r is 0.17 only. The value is considered to be a weak correlation between the population density of the state and the total listed vehicles. The population density will not the factor to affect the used vehicle market in USA.

# Which state is the most potential market?
The top 3 potential market in California, Florida, and New York State. These top 3 states are occupied one-third of the total listed vehicles. Geographically, the states located on the western and eastern coasts will be considered better potential markets than inland areas. The Pearson correlation coefficient of 0.89 between the population in 2022 and the total listed vehicles tell us the strong relationship and the states with larger population may be better potential markets but it will not be necessary to consider the population density of states.

# Section 2

# Correlation between odometer (kms run) and price of the car - Dhiraj

* Opening the dataset .csv file

* Removing the decimal from the year column

* Removing the outliers in the dataset.

* Selecting vehicles between the price of 1000 to 200,000

*  vehicles between the odometer of 100 to 900,000

In [ ]:
# Showing the relation between the odometer and the price of the listed cars

# open the csv file
clean_data = os.path.join("output_data", "vehicles_cleaned.csv")

# read the csv file
clean_df = pd.read_csv(clean_data)

# removing the decimals from the year column
clean_df['year'] = clean_df['year'].astype(int)

# removing records based on price outliers
clean_price_df = clean_df.loc[(clean_df['price'] >= 1000) & (clean_df['price'] <= 200000)]

# removing records based on Odometer outliers
cleaner_df = clean_price_df.loc[(clean_price_df['odometer'] >= 100) & (clean_price_df['odometer'] <= 900000)] 

# Getting the count of the vehicles remaing for visulaisation
# cleaner_df.count()

len(cleaner_df)

In [ ]:
clean_df.columns

# Plotting a scatter plot
* Plotting a scatter plot to show the relation between Price and Odometer of the of the car.

In [ ]:
x_axis = cleaner_df["odometer"]/1000
y_axis = cleaner_df["price"]/1000

plt.scatter(x_axis, y_axis)

plt.title("Odometer vs Price comparison")
plt.xlabel("Odometer (in thousands)")
plt.ylabel("Price (in thousands)")
plt.savefig("output_data/Odo_vs_price.png")
plt.show()

# Observations
* Higher the Odometer reading, lower the price of the car. We can see that from the graph above.


* Vehicles plotted on the botom right hand corner

    * These vehicles are very high on the their odometer reading and hence the price of the vehicle has depreciated.
    
    
* Vehicles plotted on the top left hand corner

    * These vehicles are very low on the their odometer reading and hence the price of the vehicle is high.
    
    
* Other factors to take into consideration while analysing this relation :

    * Year of manufacture is also have a bearing on the price of the vehicle

# Section 3

# The different types of vehicles listed in US from the year 2000 to 2022 - Anjana


# Summary

* The cleaned dataset of vehicles listed in the US was used for drawing conclusions about the different types of vehicles listed in US from the year 2000 to 2022.
* The number of different types of vehicles listed was found and the percentage of each type of vehicles was calculated.
* The top and bottom 5 types of vehicles listed were found out.
* The year-wise trend of number of different types of cars listed was plotted to understand how it has changed from 2000 to 2022.
* The top 3 manufacturers listed for vehicle types SUV and sedan were found out.
* The year-wise trend of maximum price of suv and sedan cars listed was plotted to understand how it has changed from 2000 to 2022.
* The price of the top 3 manufacturers of SUV and sedan was plotted to understand the maximum price of the vehicles listed.

In [ ]:
# find the number of different types of vehicles listed

type_df = vehicles_cleaned_df.groupby("type")["id"].count()

In [ ]:
# plot the different types of vehicles listed in a pie chart

labels = type_df.index.values

fig = plt.figure(figsize=(10,8))

plt.pie(type_df, labels = labels, autopct = lambda p: format(p, ".1f") if p > 3 else None, 
                    shadow = False, startangle = 140, textprops={'fontsize': 10}, rotatelabels = 18)

plt.title("Percentage of Different Types of Vehicles", pad = 32)
plt.ylabel("")

plt.legend(labels, loc = "lower right")

plt.axis("equal")
plt.tight_layout()

plt.savefig("../output_data/Percentage of Different Types of Vehicles.png")

# Observation
It can be seen that from among the 13 different types of vehicles listed, SUV and sedan holds the highest percentage with 27.1% and 28.5% respectively.

In [ ]:
# find the top 5 types of vehicles listed

top_type = type_df.sort_values(ascending = False).reset_index()
top_type.rename(columns = {"id": "count"}, inplace=True)
top_type.head(5)

In [ ]:
# find the bottom 5 types of vehicles listed

bottom_type = type_df.sort_values(ascending = True).reset_index()
bottom_type.rename(columns = {"id": "count"}, inplace = True)
bottom_type.head(5)

# Observation
The top 5 types of vehicles that are listed are: sedan, SUV, truck, pickup and hatchback and the lowest 5 types of vehicles that are listed are: bus, offroad, wagon,convertible and other type.

In [ ]:
# find the year-wise count of the different types of vehicles listed

year_df = vehicles_cleaned_df.groupby("year")["type"].value_counts()

In [ ]:
# plot the year-wise count of the different types of vehicles listed as line chart

vehicles_cleaned_df.groupby("type").year.value_counts().unstack(0).plot.line(figsize = (9.5, 8.5))
plt.title("Year-wise Count of Different Types of Vehicles")
plt.ylabel("count")
plt.tight_layout()
plt.savefig("output_data/Year-wise Count of Different Types of Vehicles.png")

# Observation
* The year-wise trend of number of different types of cars listed shows that over the years from 2000 to 2022, SUV and sedan are always the types with highest number of cars listed. 
* Also, a sharp rise in the number of cars listed can be observed in 2014. This was due to the lower gas prices in US.
* Reference: https://www.detroitnews.com/story/business/autos/2015/01/02/suvs-trucks-tow-vehicle-sales/21215945/

In [ ]:
# create a dataframe for only the top 2 types of vehicles listed - SUV and sedan

manufacturer_df = vehicles_cleaned_df[(vehicles_cleaned_df["type"].
                                       str.contains("SUV")) | 
                                      (vehicles_cleaned_df["type"].
                                       str.contains("sedan"))]
manufacturer_df = manufacturer_df[["id", "price", "type", 
                                   "manufacturer", "year"]]

In [ ]:
# find the top 3 manufacturers of SUV and sedan

top_manufacturer = manufacturer_df.groupby("type")["manufacturer"].apply(lambda x: x.value_counts().head(3)).reset_index()
top_manufacturer.rename(columns = {"level_1": "manufacturer", "manufacturer": "count"}, inplace = True)
top_manufacturer

# Observation
The top 3 manufacturers of suv are Ford, Jeep and Chevrolet and the top 3 manufacturers of sedan are Chevrolet, Toyota and Honda.

In [ ]:
# create a dataframe for only the top 3 manufacturers of SUV and sedan

manufacturer_max = manufacturer_df[((manufacturer_df["type"].str.contains("SUV") & manufacturer_df["manufacturer"].str.contains("ford"))|
                                    (manufacturer_df["type"].str.contains("SUV") & manufacturer_df["manufacturer"].str.contains("jeep"))|
                                    (manufacturer_df["type"].str.contains("SUV") & manufacturer_df["manufacturer"].str.contains("chevrolet"))|
                                    (manufacturer_df["type"].str.contains("sedan") & manufacturer_df["manufacturer"].str.contains("chevrolet"))|
                                    (manufacturer_df["type"].str.contains("sedan") & manufacturer_df["manufacturer"].str.contains("toyota"))|
                                    (manufacturer_df["type"].str.contains("sedan") & manufacturer_df["manufacturer"].str.contains("honda")))]
manufacturer_max = manufacturer_max.reset_index(drop = True)

In [ ]:
# plot the year-wise maximum price of the top 3 manufacturers of SUV and sedan as line chart

manufacturer_max.groupby(["year", "type"])["price"].max().unstack().plot.line(figsize = (9.5, 8.5))
plt.title("Year-wise Trend of Maximum Price for SUV and Sedan")
plt.ylabel("price in $")
plt.tight_layout()
plt.savefig("output_data/Year-wise Trend of Maximum Price for SUV and Sedan.png")

# Observation
* It can be observed that the maximum price of suv and sedan have been rising over the years since 2000.
* There was a sharp rise in the price in 2009 and 2014. This was because of the weak economy and falling supply of new cars.
* Reference: https://www.usatoday.com/story/money/cars/2015/02/18/record-used-car-prices-in-2014/23637775/, https://www.cnbc.com/2009/10/13/expect-used-car-prices-to-stay-high.html

In [ ]:
# find the maximum price of the top 3 manufacturers of SUV and sedan

max_p = manufacturer_max.groupby(["type","manufacturer"])["price"].max()
max_df = max_p.reset_index()

In [ ]:
# plot the price of the top 3 manufacturers of SUV and sedan as bar chart

max_p.unstack().plot.bar(figsize = (8, 7))
plt.title("Price of Top 3 Manufacturers of SUV and Sedan")
plt.ylabel("price in $")
plt.tight_layout()
plt.savefig("output_data/Price of Top 3 Manufacturers of SUV and Sedan.png")

# Observation
* It can be seen that the in the case of SUV, Ford has the maximum price listed and in the case of sedan, Chevrolet has the maximum price listed.

# Conclusions
* From among the 13 different types of vehicles listed, SUV and sedan holds the highest percentage with 27.1% and 28.5% respectively.
* The top 5 types of vehicles that are listed are: sedan, SUV, truck, pickup and hatchback and the lowest 5 types of vehicles that are listed are: bus, offroad, wagon,convertible and mini-van.
* The year-wise trend of number of different types of cars listed shows that over the years from 2000 to 2022, SUV and sedan are always the types with highest number of cars listed. Also, a sharp rise in the number of cars listed can be observed in 2014. This was due to the lower gas prices in US.
* The top 3 manufacturers of suv are Ford, Jeep and Chevrolet and the top 3 manufacturers of sedan are Chevrolet, Toyota and Honda.
* The maximum price of suv and sedan have been rising over the years since 2000. There was a sharp rise in the price in 2009 and 2014. This was because of the weak economy and falling supply of new cars.
* In the case of SUV, Ford has the maximum price listed and in the case of sedan, Chevrolet has the maximum price listed.

# Section 4

# Observations based on comparing the Color of the Vehicles listed with the number of listings, prices of listings and manufacturer – Deon

# Analysis
Observations based on comparing the Color of the Vehicles listed with the number of listings, prices of listings and manufacturer.

In [ ]:
# find the number of vehicles with different paint color
colors_df = vehicles_cleaned_df['paint_color'].value_counts()
colors_df

In [ ]:
# plot a bar chart for the number of vehicles with different paint color

colors_df.plot(kind='bar')
plt.title("Number of Listings per Paint Color")
plt.xlabel("Paint Color")
plt.ylabel("Number of Cars")
plt.xlim(-1,len(colors_df.index))
plt.savefig("output_data/Number of Listings per Paint Color.png")
plt.show()

# Observations
* White Color Vehicles are the most listed with 16,299 listings.
* Followed by Black(12,026), Silver(10,242), Grey(8528), Blue(6959) and Red(6126) Color Vehicles.
* The Vehicle Color with the fewest listings is Purple with 161 listings.

In [ ]:
# find the sum of price of different paint colors
colors_price_df = vehicles_cleaned_df.groupby(['paint_color'])['price'].sum()/1000000
colors_price_df

In [ ]:
# plot a bar chart for the sum of price of different paint colors
colors_price_df.plot(kind='bar')
plt.title("Sum of Prices of Listings per Paint Color")
plt.xlabel("Paint Color")
plt.ylabel("Sum of Prices of Listings (in Millions of Dollars)")
plt.xlim(-1,len(colors_price_df.index))
plt.savefig("output_data/Sum of Prices of Listings per Paint Color.png")
plt.show()

# Observations
* White Color Vehicles had the largest sum price of listings (USD 295.6 million).
* Followed by Black(USD 190.3 million), Silver(USD 119.5 million), Grey(USD 119.2 million), Red(USD 84.8 million) and Blue(USD 82.6 million) Color Vehicles.
* Even though there are 833 more Blue Vehicles listed than Red Vehicles, the sum price of listings of Red Vehicles is greater than that of Blue Vehicles by USD 2.3 million.
* The Vehicle Color with smallest sum price of listings is Purple with USD 1.6 million.

In [ ]:
# find the number of manufacturers listed for white paint color
white_cleaned_df = vehicles_cleaned_df.loc[vehicles_cleaned_df['paint_color']=='white']
manufacturer_white_df = white_cleaned_df.groupby(['manufacturer'])['paint_color'].count()
manufacturer_white_df

In [ ]:
# create a bar chart for the number of manufacturers listed for white paint color
manufacturer_white_df.plot(kind='bar',figsize=(12,6))
plt.title("Number of White Vehicles Listed per Manufacturer")
plt.xlabel("Manufacturer")
plt.ylabel("Number of White Vehicles")
plt.xlim(-1,len(manufacturer_white_df.index))
plt.savefig("output_data/Number of White Vehicles Listed per Manufacturer.png")
plt.show()

# Observations
* The Vehicle Manufacturer with the largest number of White Vehicles listed is Ford with 5146 listings.
* The second largest is Chevrolet with 2836 White Vehicles Listed.
* Both these manufacturers are American owned.
* The third largest is Toyota with 1095 White Vehicles Listed. (Japanese Owned)

# Section 5

# What is the most profitable way to sell cars online? - Tim
* This section explores the saturation and volatility in the market.

In [ ]:
# Removing car prices that are 1 or less and also removing any prices that are greater
# the 95% quartile or lower than the 5% quartile
vehicle_listing_df = vehicles_cleaned_df.loc[vehicles_cleaned_df["price"] > 1]
vehicle_listing_df = vehicles_cleaned_df[vehicles_cleaned_df.groupby("manufacturer").price.\
      transform(lambda x : (x<x.quantile(0.95))&(x>(x.quantile(0.05)))).eq(1)]
vehicle_listing_df.max()

In [ ]:
# Calculating the number of cars listed
num_cars = vehicle_listing_df.groupby("manufacturer")["id"].count()
num_cars = num_cars.sort_values(ascending=False)
num_cars.rename("count").reset_index()

In [ ]:
num_cars.plot(kind="pie", rotatelabels=18, figsize=(15,10), autopct=lambda p: format(p, ".1f") if p > 1.5 else None,
             shadow = False, startangle = 140, textprops={'fontsize': 10})

labels = num_cars.index.values

plt.title("Percentage of Different Manufactuers", pad = 40)
plt.ylabel("")

plt.legend(labels, loc = "lower right")

plt.axis("equal")
plt.tight_layout()

# Observation
It can be seen that fords and chervolet are the most listed cars, with them making up 19.3% and 14.2% of the listed used cars

In [ ]:
# Average price per car manufacturer
average_listing_price = vehicle_listing_df.groupby(["manufacturer"])["price"].mean().sort_index()
average_listing_price.plot.bar()
plt.title("Average Price Per Car Manufacturer")
plt.ylabel("price in $")
plt.savefig("output_data/Average Price Per Car Manufacturer.png")

In [ ]:
average_listing_price[["ford", "chevrolet", "toyota"]].reset_index()

In [ ]:
# Top 5 average costing cars
top_avg_list_price = average_listing_price.sort_values(ascending=False).reset_index()
top_avg_list_price.rename(columns = {"price":"average price"}, inplace=True)
top_avg_list_price.head(5)

In [ ]:
# Bottom 5 average costing cars
bot_avg_list_price = average_listing_price.sort_values(ascending=True).reset_index()
bot_avg_list_price.rename(columns = {"price":"average price"}, inplace=True)
bot_avg_list_price.head(5)

# Observation
* We can observe that the though ford, chevrolet and toyota have the highest number of listings, they still have a relatively high average price. This indicates greater price stability amongst those manufacturers.


# Finding volatility
* Since we will be sourcing used cars, we need to be buy them for as cheap as possible and sell them for as high as possible to make the most profit

In [ ]:
iqr = vehicle_listing_df.boxplot(column="price", by="manufacturer", rot=90, figsize=(15,10))
plt.savefig("output_data/Boxplot Grouped by Manufacturer.png")

In [ ]:
# iqr = vehicle_listing_df.quantile([0.75, 0.25]).groupby("manufacturer")["price"].agg(np.substract.reduce)

grouper = vehicle_listing_df.groupby("manufacturer")
q1, q3 = grouper.quantile(0.25), grouper.quantile(0.75)
iqr = q3 - q1
iqr = iqr.rename(columns={"price": "IQR"})

iqr["IQR"].sort_values(ascending=False).reset_index()